# Predicting Teacher Turnover

__Meaghan Ross__

Flatiron School Capstone

### Data Preparation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ls data/

2017-2018 Enrollment & Demographics.xlsx
SPR_SY1718_School_Metric_Scores_20190129.xlsx
df_2019.csv
df_eda.csv
employee_information_April2018.csv
employee_information_April2019.csv


#### Teacher Data

Load in the data from the 2017-2018 and 2018-2019 employee information files.

In [3]:
teacher_df_2018 = pd.read_csv('data/employee_information_April2018.csv')
teacher_df_2019 = pd.read_csv('data/employee_information_April2019.csv')

In [4]:
teacher_df_2018.head()

,LAST_NAME,FIRST_NAME,PAY_RATE_TYPE,PAY_RATE,TITLE_DESCRIPTION,HOME_ORGANIZATION,HOME_ORGANIZATION_DESCRIPTION,ORGANIZATION_LEVEL,TYPE_OF_REPRESENTATION,GENDER,RUN_DATE
0,AARON,ANDREA,SALARIED,32199,"GENERAL CLEANER, 8 HOURS",4300,"HESTON, EDWARD SCHOOL",ELEMENTARY SCHOOL,LOCAL 1201,F,4/1/2018
1,AARON,PEGGY,SALARIED,10084,"STUDENT CLIMATE STAFF,4 HOURS",6360,ROOSEVELT ELEMENTARY SCHOOL,ELEMENTARY SCHOOL,LOCAL 634,F,4/1/2018
2,ABARY,RODNEY,SALARIED,76461,SCHOOL NURSE,2370,"MCDANIEL, DELAPLAINE SCHOOL",ELEMENTARY SCHOOL,PFT-TEACHER,M,4/1/2018
3,ABATE,JO-ANN,HOURLY,39,TEACHER-EXTRA CURR/STAFF DEVEL,9EW0,NON-PUBLIC PROGRAMS,NON ADMINISTRATIVE OFFICE,PFT-TEACHER,F,4/1/2018
4,ABAYOMI-IGE,OLABIMPE,SALARIED,90051,"TEACHER,SPEC EDUCATION",5070,PARKWAY-NORTHWEST HIGH SCHOOL,HIGH SCHOOL,PFT-TEACHER,F,4/1/2018


In [5]:
teacher_df_2019.head()

,LAST_NAME,FIRST_NAME,PAY_RATE_TYPE,PAY_RATE,TITLE_DESCRIPTION,HOME_ORGANIZATION,HOME_ORGANIZATION_DESCRIPTION,ORGANIZATION_LEVEL,TYPE_OF_REPRESENTATION,GENDER,RUN_DATE
0,AARAS,YOUSRA,SALARIED,10383,"STUDENT CLIMATE STAFF,4 HOURS",2620,ACADEMY AT PALUMBO,HIGH SCHOOL,LOCAL 634,F,4/1/2019
1,AARON,ANDREA,SALARIED,33165,"GENERAL CLEANER, 8 HOURS",4300,"HESTON, EDWARD SCHOOL",ELEMENTARY SCHOOL,LOCAL 1201,F,4/1/2019
2,AARON,ATIA,SALARIED,12979,"STUDENT CLIMATE STAFF,5 HOURS",2050,SCIENCE LEADERSHIP ACADEMY MS,MIDDLE SCHOOL,LOCAL 634,F,4/1/2019
3,AARON,PEGGY,SALARIED,10383,"STUDENT CLIMATE STAFF,4 HOURS",7130,"WAGNER, GEN. LOUIS MIDDLE SCH.",MIDDLE SCHOOL,LOCAL 634,F,4/1/2019
4,ABARA,BERNADINE,SALARIED,7788,"STUDENT CLIMATE STAFF,3 HOURS",8310,"MOORE, J. HAMPTON SCHOOL",ELEMENTARY SCHOOL,LOCAL 634,F,4/1/2019


In [6]:
teacher_df_2018.shape

(19181, 11)

In [7]:
teacher_df_2019.shape

(20151, 11)

In order to find out the teacher turnover  for the school years following  the 2017-2018 school year, we will merge each school year's personnel files with the 2018-2019 school year's personnel files using a left join to identify all of the employees of the Philadelphia school system in the given year. The join will use the employee's first and last name as well as their school name since we want to identify teachers who remained in the same school the following year. If there are null values in the following school year, that indicates that the employee did not return for the following school year.

In [8]:
teacher_turnover_2019 = pd.merge(teacher_df_2018, teacher_df_2019, 
                                 on=['LAST_NAME', 'FIRST_NAME', 'HOME_ORGANIZATION'], 
                                 how='left', suffixes=('_2018', '_2019'))

In [9]:
teacher_turnover_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19279 entries, 0 to 19278
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           19278 non-null  object 
 1   FIRST_NAME                          19279 non-null  object 
 2   PAY_RATE_TYPE_2018                  19279 non-null  object 
 3   PAY_RATE_2018                       19279 non-null  int64  
 4   TITLE_DESCRIPTION_2018              19279 non-null  object 
 5   HOME_ORGANIZATION                   19279 non-null  object 
 6   HOME_ORGANIZATION_DESCRIPTION_2018  19279 non-null  object 
 7   ORGANIZATION_LEVEL_2018             19279 non-null  object 
 8   TYPE_OF_REPRESENTATION_2018         18962 non-null  object 
 9   GENDER_2018                         19279 non-null  object 
 10  RUN_DATE_2018                       19279 non-null  object 
 11  PAY_RATE_TYPE_2019                  14258

The merged DataFrames include all employes of the School District of Philadelphia. Let's look at identifying only those who are labeled as teachers.

In [10]:
teacher_turnover_2019['TITLE_DESCRIPTION_2018'].value_counts()

TEACHER,FULL TIME                 6789
SPECIAL EDUCATION ASSISTANT       1848
TEACHER,SPEC EDUCATION            1389
GENERAL CLEANER, 8 HOURS           558
FOOD SVCS ASSISTANT                428
                                  ... 
PROG MGR,INTER&TRANS/ATTEND&TR       1
BROAD RESIDENT                       1
EXECUTIVE SECRETARY,BILINGUAL        1
SVP,FOOD SERVICES                    1
PROJECT COORD,INFO TECHNOLOGY        1
Name: TITLE_DESCRIPTION_2018, Length: 591, dtype: int64

We are going to create a new DataFrame that only includes those employees identified as `TEACHER,FULL TIME` or `TEACHER,SPEC EDUCATION` as the remaining employee types are outside of the classroom or support staff who do not need to be liscenced educators.

In [11]:
teacher_turnover_2019 = teacher_turnover_2019.loc[(teacher_turnover_2019['TITLE_DESCRIPTION_2018']=='TEACHER,FULL TIME')|
                           (teacher_turnover_2019['TITLE_DESCRIPTION_2018']=='TEACHER,SPEC EDUCATION')]


In [12]:
teacher_turnover_2019.isna().sum()

LAST_NAME                                1
FIRST_NAME                               0
PAY_RATE_TYPE_2018                       0
PAY_RATE_2018                            0
TITLE_DESCRIPTION_2018                   0
HOME_ORGANIZATION                        0
HOME_ORGANIZATION_DESCRIPTION_2018       0
ORGANIZATION_LEVEL_2018                  0
TYPE_OF_REPRESENTATION_2018              0
GENDER_2018                              0
RUN_DATE_2018                            0
PAY_RATE_TYPE_2019                    1593
PAY_RATE_2019                         1593
TITLE_DESCRIPTION_2019                1593
HOME_ORGANIZATION_DESCRIPTION_2019    1593
ORGANIZATION_LEVEL_2019               1593
TYPE_OF_REPRESENTATION_2019           1593
GENDER_2019                           1593
RUN_DATE_2019                         1593
dtype: int64

Looking at the null values now, we can see there is one teacher with a null value for their last name, so we will fill it with 'None'. The additional null values are all in the columns from the following school year which indicates records of teachers who did not stay for the following school year, and we want to keep those null values for now.

In [13]:
teacher_turnover_2019.loc[:,['LAST_NAME']]=teacher_turnover_2019['LAST_NAME'].fillna(value='None')

In [14]:
teacher_turnover_2019.isna().sum()

LAST_NAME                                0
FIRST_NAME                               0
PAY_RATE_TYPE_2018                       0
PAY_RATE_2018                            0
TITLE_DESCRIPTION_2018                   0
HOME_ORGANIZATION                        0
HOME_ORGANIZATION_DESCRIPTION_2018       0
ORGANIZATION_LEVEL_2018                  0
TYPE_OF_REPRESENTATION_2018              0
GENDER_2018                              0
RUN_DATE_2018                            0
PAY_RATE_TYPE_2019                    1593
PAY_RATE_2019                         1593
TITLE_DESCRIPTION_2019                1593
HOME_ORGANIZATION_DESCRIPTION_2019    1593
ORGANIZATION_LEVEL_2019               1593
TYPE_OF_REPRESENTATION_2019           1593
GENDER_2019                           1593
RUN_DATE_2019                         1593
dtype: int64

#### School Data

Load in the data from the the 2017-2018 School Progress Reports from the School District of Philadelphia. This data includes information about achievement, progress, and climate scores as well as attendance and survey data.

In [15]:
school_df_2018 = pd.read_excel('data/SPR_SY1718_School_Metric_Scores_20190129.xlsx', sheet_name='SPR SY2017-2018')
school_df_2018.head()

,School,SRC School ID,ULCS Code,Report,Rpt Type Long,Street Address,City,State,Zip Code,Phone Number,...,FAFSA Tier,Student Survey College & Career Score,Student Survey College & Career Pts Earn,Student Survey College & Career Pts Poss,Student Survey College & Career Pct Earn,Student Survey College & Career Tier,Teach Effect Distinguished Score,Teach Effect Instruction Score,Teacher Attendance Score,Student Survey Teaching Score
0,John Bartram High School,101,1010,HS,High School,2401 S. 67th St.,Philadelphia,PA,19142,215-400-8100,...,INTERVENE,33,0.33,1,33,WATCH,Data Not Available,Data Not Available,52,43
1,West Philadelphia High School,102,1020,HS,High School,4901 Chestnut St.,Philadelphia,PA,19139,215-400-7900,...,WATCH,0,0,1,0,INTERVENE,Data Not Available,Data Not Available,49,0
2,High School of the Future,103,1030,HS,High School,4021 Parkside Ave.,Philadelphia,PA,19104,215-400-7790,...,REINFORCE,40,0.4,1,40,WATCH,Data Not Available,Data Not Available,56,36
3,Paul Robeson High School for Human Services,105,1050,HS,High School,4125 Ludlow St.,Philadelphia,PA,19104,215-400-7780,...,MODEL,39,0.39,1,39,WATCH,Data Not Available,Data Not Available,68,53
4,William L. Sayre High School,110,1100,HS,High School,5800 Walnut St.,Philadelphia,PA,19139,215-400-7800,...,INTERVENE,0,0,1,0,INTERVENE,Data Not Available,Data Not Available,71,0


The School Progress Reports contain many features, including the way some of the features are calculated and features that are specific to a certain type of school and are not reported across all schools. For the purpose of our model, we want to include information that is reported for all schools including features regarding student achievement, school climate, and progress from prior years.

In [16]:
school_col_keep = ['School', 'SRC School ID', 'ULCS Code', 'Zip Code', 'Report', 
                   'Turnaround Model', 'Enrollment', 'Grades Served', 
                   'Admissions Type', 'Overall Score', 'Overall Tier', 
                   'Ach Score', 'Ach Tier', 'Prog Score', 'Prog Tier', 
                   'Clim Score', 'Clim Tier', 'Attendance (95%+) Score', 
                   'Attendance (95%+) Tier', 'Retention Score', 'Retention Tier', 
                   'ISS Score', 'ISS Tier', 'OSS Score', 'OSS Tier', 
                   'Student Survey Climate Score', 'Student Survey Climate Tier', 
                   'Parent Survey Climate Score', 'Parent Survey Climate Tier', 
                   'Parent Survey Participation Score', 'Parent Survey Participation Tier', 
                   'Teacher Attendance Score','Student Survey Teaching Score']


In [17]:
school_df_2018=school_df_2018[school_col_keep]

In [18]:
school_df_2018.head()

,School,SRC School ID,ULCS Code,Zip Code,Report,Turnaround Model,Enrollment,Grades Served,Admissions Type,Overall Score,...,OSS Score,OSS Tier,Student Survey Climate Score,Student Survey Climate Tier,Parent Survey Climate Score,Parent Survey Climate Tier,Parent Survey Participation Score,Parent Survey Participation Tier,Teacher Attendance Score,Student Survey Teaching Score
0,John Bartram High School,101,1010,19142,HS,NaN,561,'9-12',Neighborhood,13,...,82,WATCH,52,REINFORCE,0,INTERVENE,2,INTERVENE,52,43
1,West Philadelphia High School,102,1020,19139,HS,Turnaround,478,'9-12',Neighborhood,8,...,76,INTERVENE,0,INTERVENE,0,INTERVENE,8,INTERVENE,49,0
2,High School of the Future,103,1030,19104,HS,NaN,488,'9-12',Citywide,25,...,87,REINFORCE,55,REINFORCE,0,INTERVENE,2,INTERVENE,56,36
3,Paul Robeson High School for Human Services,105,1050,19104,HS,NaN,299,'9-12',Citywide,44,...,95,MODEL,64,REINFORCE,82,MODEL,13,INTERVENE,68,53
4,William L. Sayre High School,110,1100,19139,HS,NaN,425,'9-12',Neighborhood,8,...,82,WATCH,0,INTERVENE,0,INTERVENE,8,INTERVENE,71,0


When checking for null values, the `Turnaround Model` has null values meaning the school was not identified to participate in a turnaround model for a given school year. The null values will be filled with 'None'.

In [19]:
school_df_2018.isna().sum()

School                                 0
SRC School ID                          0
ULCS Code                              0
Zip Code                               0
Report                                 0
Turnaround Model                     272
Enrollment                             0
Grades Served                          0
Admissions Type                        0
Overall Score                          0
Overall Tier                           0
Ach Score                              0
Ach Tier                               0
Prog Score                             0
Prog Tier                              0
Clim Score                             0
Clim Tier                              0
Attendance (95%+) Score                0
Attendance (95%+) Tier                 0
Retention Score                        0
Retention Tier                         0
ISS Score                              0
ISS Tier                               0
OSS Score                              0
OSS Tier        

In [20]:
school_df_2018.loc[:,['Turnaround Model']]=school_df_2018['Turnaround Model'].fillna(value='None')

In [21]:
school_df_2018.isna().sum()

School                               0
SRC School ID                        0
ULCS Code                            0
Zip Code                             0
Report                               0
Turnaround Model                     0
Enrollment                           0
Grades Served                        0
Admissions Type                      0
Overall Score                        0
Overall Tier                         0
Ach Score                            0
Ach Tier                             0
Prog Score                           0
Prog Tier                            0
Clim Score                           0
Clim Tier                            0
Attendance (95%+) Score              0
Attendance (95%+) Tier               0
Retention Score                      0
Retention Tier                       0
ISS Score                            0
ISS Tier                             0
OSS Score                            0
OSS Tier                             0
Student Survey Climate Sc

__Enrollment & Demographics Data__

Load in the data from the 2017-2018 school year. This data includes school level data regarding the English Language Learners (ELL), Individualized Education Plans (IEP), gender, ethnicity, and economically disadvantaged rates in each school.

In [22]:
ls data/

2017-2018 Enrollment & Demographics.xlsx
SPR_SY1718_School_Metric_Scores_20190129.xlsx
df_2019.csv
df_eda.csv
employee_information_April2018.csv
employee_information_April2019.csv


In [23]:
df_ell = pd.read_excel('data/2017-2018 Enrollment & Demographics.xlsx', 
                       sheet_name='ELL', header=5)
df_iep = pd.read_excel('data/2017-2018 Enrollment & Demographics.xlsx', 
                       sheet_name='IEP', header=5)
df_gender = pd.read_excel('data/2017-2018 Enrollment & Demographics.xlsx', 
                       sheet_name='Gender', header=5)
df_ethnicity = pd.read_excel('data/2017-2018 Enrollment & Demographics.xlsx', 
                       sheet_name='Ethnicity', header=5)
df_ed = pd.read_excel('data/2017-2018 Enrollment & Demographics.xlsx', 
                       sheet_name='Economically Disadvantaged', header=5)


In [24]:
df_ell=df_ell[df_ell['Grade']=='All Grades']
df_ell.rename(columns = {'Percent %':'Percent ELL'}, inplace = True)
df_ell = df_ell[['School ID','Percent ELL']]

In [25]:
df_iep=df_iep[df_iep['Grade']=='All Grades']
df_iep.rename(columns = {'Percent %':'Percent IEP'}, inplace = True)
df_iep = df_iep[['School ID','Percent IEP']]

In [26]:
df_gender=df_gender[df_gender['Grade']=='All Grades']
df_gender.rename(columns = {'Percent %.1':'Percent Male'}, inplace = True)
df_gender = df_gender[['School ID','Percent Male']]

In [27]:
df_ethnicity=df_ethnicity[df_ethnicity['Grade']=='All Grades']
df_ethnicity.rename(columns = {'Percent %.1':'Percent Asian',
                               'Percent %.2':'Percent Black/African American',
                               'Percent %.3':'Percent Hispanic',
                               'Percent %.4':'Percent Multi-Race',
                               'Percent %.6':'Percent White'}, inplace = True)
df_ethnicity = df_ethnicity[['School ID',
                             'Percent Asian', 
                             'Percent Black/African American', 
                             'Percent Hispanic', 
                             'Percent Multi-Race', 
                             'Percent White']]

In [28]:
df_ed=df_ed[df_ed['Grade']=='All Grades']
df_ed.rename(columns = {'CEP Economically Disadvantaged\n Rate':'Economically Disadvantaged Rate'}, inplace = True)
df_ed = df_ed[['School ID','Economically Disadvantaged Rate']]


In [29]:
dfs = [df.set_index(['School ID']) for df in [df_ell, df_iep, df_gender, df_ethnicity, df_ed]]

school_demographics = pd.concat(dfs, axis=1).reset_index()

In [30]:
school_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   School ID                        215 non-null    int64  
 1   Percent ELL                      215 non-null    float64
 2   Percent IEP                      215 non-null    float64
 3   Percent Male                     215 non-null    float64
 4   Percent Asian                    215 non-null    float64
 5   Percent Black/African American   215 non-null    float64
 6   Percent Hispanic                 215 non-null    float64
 7   Percent Multi-Race               215 non-null    float64
 8   Percent White                    215 non-null    float64
 9   Economically Disadvantaged Rate  214 non-null    float64
dtypes: float64(9), int64(1)
memory usage: 16.9 KB


In [31]:
school_demographics.shape

(215, 10)

__Merge School Progress Reports with School Demographics__

In [32]:
school_df_2018.shape

(321, 33)

In [33]:
school_df_2018['SRC School ID']

0      101
1      102
2      103
3      105
4      110
      ... 
316    X28
317    X40
318    X41
319    X42
320    X43
Name: SRC School ID, Length: 321, dtype: object

In [34]:
school_df_2018.loc[:,['SRC School ID']] = pd.to_numeric(school_df_2018['SRC School ID'], 
                                                        errors='coerce')

In [35]:
school_df_2018.dropna(subset = ['SRC School ID'], inplace=True)

In [36]:
school_df_2018.shape

(217, 33)

In [37]:
full_school_2018 = pd.merge(school_df_2018, school_demographics, 
                            left_on='SRC School ID', 
                            right_on='School ID',
                            how='left')

In [38]:
full_school_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 43 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   School                             217 non-null    object 
 1   SRC School ID                      217 non-null    float64
 2   ULCS Code                          217 non-null    int64  
 3   Zip Code                           217 non-null    int64  
 4   Report                             217 non-null    object 
 5   Turnaround Model                   217 non-null    object 
 6   Enrollment                         217 non-null    int64  
 7   Grades Served                      217 non-null    object 
 8   Admissions Type                    217 non-null    object 
 9   Overall Score                      217 non-null    object 
 10  Overall Tier                       217 non-null    object 
 11  Ach Score                          217 non-null    object 

### Merging Teacher Data with School Data

In order to merge the individual teacher data with the school progress report data, we will need to merge on the school code as the school name is not written the same between files. The school code is consistent, however, in the teacher data it is stored as an object when it is numerical in the school data.

We will turn the `HOME_ORGANIZATION` code into a numeric and those that are not numerical will be coerced to a null value. The staff with the non-numeric codes are assigned to locations that are not actual school buildings such as code `9KT0` represents the Office of High School Support.

Then, we will drop the null values from the `HOME ORGANIZATION`.

In [39]:
teacher_turnover_2019.loc[:,['HOME_ORGANIZATION']] = pd.to_numeric(teacher_turnover_2019['HOME_ORGANIZATION'], 
                                                        errors='coerce')

In [40]:
teacher_turnover_2019.dropna(subset = ['HOME_ORGANIZATION'], inplace=True)

In [41]:
teacher_turnover_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8085 entries, 4 to 19277
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           8085 non-null   object 
 1   FIRST_NAME                          8085 non-null   object 
 2   PAY_RATE_TYPE_2018                  8085 non-null   object 
 3   PAY_RATE_2018                       8085 non-null   int64  
 4   TITLE_DESCRIPTION_2018              8085 non-null   object 
 5   HOME_ORGANIZATION                   8085 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  8085 non-null   object 
 7   ORGANIZATION_LEVEL_2018             8085 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         8085 non-null   object 
 9   GENDER_2018                         8085 non-null   object 
 10  RUN_DATE_2018                       8085 non-null   object 
 11  PAY_RATE_TYPE_2019                  6521 n

Now we can merge the teacher data with the school data using what is referred to as `HOME_ORGANIZATION` in the teacher data and the `ULCS Code` for the school data.

In [42]:
full_turnover_2019= pd.merge(teacher_turnover_2019, full_school_2018, 
                             left_on='HOME_ORGANIZATION', 
                             right_on='ULCS Code',
                             how='left')

In [43]:
full_turnover_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8252 entries, 0 to 8251
Data columns (total 62 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           8252 non-null   object 
 1   FIRST_NAME                          8252 non-null   object 
 2   PAY_RATE_TYPE_2018                  8252 non-null   object 
 3   PAY_RATE_2018                       8252 non-null   int64  
 4   TITLE_DESCRIPTION_2018              8252 non-null   object 
 5   HOME_ORGANIZATION                   8252 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  8252 non-null   object 
 7   ORGANIZATION_LEVEL_2018             8252 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         8252 non-null   object 
 9   GENDER_2018                         8252 non-null   object 
 10  RUN_DATE_2018                       8252 non-null   object 
 11  PAY_RATE_TYPE_2019                  6663 no

In [44]:
full_turnover_2019[full_turnover_2019.filter(like='School').isnull().any(1)]

,LAST_NAME,FIRST_NAME,PAY_RATE_TYPE_2018,PAY_RATE_2018,TITLE_DESCRIPTION_2018,HOME_ORGANIZATION,HOME_ORGANIZATION_DESCRIPTION_2018,ORGANIZATION_LEVEL_2018,TYPE_OF_REPRESENTATION_2018,GENDER_2018,...,School ID,Percent ELL,Percent IEP,Percent Male,Percent Asian,Percent Black/African American,Percent Hispanic,Percent Multi-Race,Percent White,Economically Disadvantaged Rate
14,ABRAHAM-CUFF,NAFHRAH,SALARIED,48527,"TEACHER,SPEC EDUCATION",6400.0,WIDENER MEMORIAL SCHOOL,ELEMENTARY SCHOOL,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,ABT,DEBBIE,SALARIED,78376,"TEACHER,FULL TIME",6392.0,STEEL SCHOOL HEAD START,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,ALLEN,HEATHER,SALARIED,52196,"TEACHER,FULL TIME",6202.0,"DAY, ANNA B. HEAD START",EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,ALLEN,MAYA,SALARIED,45359,"TEACHER,FULL TIME",6341.0,PENNELL HEAD START,EARLY CHILDHOOD,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,ALLMAN,ELIZABETH,SALARIED,59532,"TEACHER,FULL TIME",4399.0,WRIGHT HEAD START,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8167,YORKONIS,TRACY,SALARIED,67789,"TEACHER,FULL TIME",1475.0,HAVERFORD CTR BRIGHT FUTURES,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8185,YU,JUYEON,SALARIED,90051,"TEACHER,FULL TIME",6400.0,WIDENER MEMORIAL SCHOOL,ELEMENTARY SCHOOL,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8197,ZAJDEL,HEATHER,SALARIED,90051,"TEACHER,FULL TIME",8460.0,PHILA LEARNING ACADEMY-SOUTH,TRANSITION / OVERAGE SCHOOL,PFT-TEACHER,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8213,ZELNER,KATHLEEN,SALARIED,69623,"TEACHER,FULL TIME",8272.0,HOLME HEAD START,EARLY CHILDHOOD,PFT- PRE K,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After the merge, we can see that the teachers with no school information listed are those teachers who work in schools that did not participate in the School Progress Report, such as Head Start programs, Transition/Overage Schools, or schools that serve significant physical or medical disabilities. As we don't have School Progress Report data for those schools, we will drop those rows which represent minimal proportions of the data.

In [45]:
full_turnover_2019.dropna(subset = ['School'], inplace=True)

In [46]:
full_turnover_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7984 entries, 0 to 8251
Data columns (total 62 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           7984 non-null   object 
 1   FIRST_NAME                          7984 non-null   object 
 2   PAY_RATE_TYPE_2018                  7984 non-null   object 
 3   PAY_RATE_2018                       7984 non-null   int64  
 4   TITLE_DESCRIPTION_2018              7984 non-null   object 
 5   HOME_ORGANIZATION                   7984 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  7984 non-null   object 
 7   ORGANIZATION_LEVEL_2018             7984 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         7984 non-null   object 
 9   GENDER_2018                         7984 non-null   object 
 10  RUN_DATE_2018                       7984 non-null   object 
 11  PAY_RATE_TYPE_2019                  6450 no

We only want to keep teachers who had a salary listed for any given year, so we filter out the teachers listed as having a salary equal to 0, which is a minimal proportion of the data.

In [47]:
full_turnover_2019 = full_turnover_2019[full_turnover_2019['PAY_RATE_2018']>0]

In [48]:
full_turnover_2019.shape

(7927, 62)

We also want to create a dummy column that indicates whether a teacher is new to teaching or not using their salary information. New to teaching will be those teachers who are qualified for the Teacher Induction program:
>"Our Induction program provides one year of support that prioritizes skill development, personal reflection, and professional networking. Our model provides teachers with general pedagogical strategies that are designed for teachers in years 0-3." from https://www.philasd.org/teachingandlearning/professional-development/induction/

That means we will use cutoffs up to Step 3 from the Philadelphia Federation of Teachers' [Salary Schedule](https://jobs.philasd.org/wp-content/uploads/sites/47/2018/08/PFT-Salary-Schedules.pdf) that was in place from 2013-2021 to indicate new to teaching. 


In [49]:
#create new column and set all values to 0
full_turnover_2019['NEW_TEACHER']=0

In [50]:
conditions_2019 = [
    (full_turnover_2019['PAY_RATE_2018'] <= 53281)&(full_turnover_2019['TITLE_DESCRIPTION_2018'] == 'TEACHER,FULL TIME'),
    (full_turnover_2019['PAY_RATE_2018'] <= 54534)&(full_turnover_2019['TITLE_DESCRIPTION_2018'] == 'TEACHER,SPEC EDUCATION')
    ]
values = [1, 1]
full_turnover_2019['NEW_TEACHER'] = np.select(conditions_2019, values)

In [51]:
full_turnover_2019['NEW_TEACHER'].value_counts(normalize=True)

0    0.828561
1    0.171439
Name: NEW_TEACHER, dtype: float64

We will establish a `TURNOVER` column that will act as the target column for modeling purposes. To do this, if a record has non-null values from the following year's dataset, then they will be marked as 0 indicating they were retained in their school from the prior year. However, if a record has a null value in the following year's dataset, they will be marked with a 1 indicating they turned over from their school following the year prior.

In [52]:
full_turnover_2019['TURNOVER']=np.where(full_turnover_2019['PAY_RATE_2019'].isnull(), 1,0)

In [53]:
full_turnover_2019['TURNOVER'].value_counts(normalize=True)

0    0.810647
1    0.189353
Name: TURNOVER, dtype: float64

From this, we can see that the turnover rate ranges from about 18% to nearly 22% of the public school teachers in Philadelphia from 2017-2019. This does indicate that there is a class imbalance at play that will need to be accounted for when modeling.

We still need to clean up some of the data before modeling. We can drop columns that will not be helpful for modeling, such as the teacher data from the following year in each DataFrame since it contains null values from those teachers that left their school.

In [54]:
full_turnover_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7927 entries, 0 to 8251
Data columns (total 64 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   LAST_NAME                           7927 non-null   object 
 1   FIRST_NAME                          7927 non-null   object 
 2   PAY_RATE_TYPE_2018                  7927 non-null   object 
 3   PAY_RATE_2018                       7927 non-null   int64  
 4   TITLE_DESCRIPTION_2018              7927 non-null   object 
 5   HOME_ORGANIZATION                   7927 non-null   float64
 6   HOME_ORGANIZATION_DESCRIPTION_2018  7927 non-null   object 
 7   ORGANIZATION_LEVEL_2018             7927 non-null   object 
 8   TYPE_OF_REPRESENTATION_2018         7927 non-null   object 
 9   GENDER_2018                         7927 non-null   object 
 10  RUN_DATE_2018                       7927 non-null   object 
 11  PAY_RATE_TYPE_2019                  6426 no

In [55]:
cols_to_drop_2019 = ['PAY_RATE_TYPE_2019', 'PAY_RATE_2019','TITLE_DESCRIPTION_2019', 
                'HOME_ORGANIZATION_DESCRIPTION_2019', 'ORGANIZATION_LEVEL_2019',
                'TYPE_OF_REPRESENTATION_2019','GENDER_2019','RUN_DATE_2019', 'RUN_DATE_2018',
                'HOME_ORGANIZATION_DESCRIPTION_2018', 'PAY_RATE_TYPE_2018', 'TYPE_OF_REPRESENTATION_2018',
                'ULCS Code', 'Report', 'School ID', 'SRC School ID','Grades Served']

In [56]:
full_turnover_2019.drop(cols_to_drop_2019, axis=1, inplace=True)

In [57]:
full_turnover_2019.head()

,LAST_NAME,FIRST_NAME,PAY_RATE_2018,TITLE_DESCRIPTION_2018,HOME_ORGANIZATION,ORGANIZATION_LEVEL_2018,GENDER_2018,School,Zip Code,Turnaround Model,...,Percent IEP,Percent Male,Percent Asian,Percent Black/African American,Percent Hispanic,Percent Multi-Race,Percent White,Economically Disadvantaged Rate,NEW_TEACHER,TURNOVER
0,ABAYOMI-IGE,OLABIMPE,90051,"TEACHER,SPEC EDUCATION",5070.0,HIGH SCHOOL,F,Parkway Northwest High School,19138.0,None,...,0.103846,0.488462,0.000000,0.934615,0.046154,0.007692,0.011538,0.824615,0,0
1,ABBOTT,JOYCE,76461,"TEACHER,FULL TIME",1290.0,ELEMENTARY SCHOOL,F,Andrew Hamilton School,19139.0,None,...,0.194757,0.558052,0.001873,0.893258,0.014981,0.069288,0.014981,1.000000,0,0
2,ABDUL-LATEEF,VILLIA,56531,"TEACHER,FULL TIME",1010.0,HIGH SCHOOL,F,John Bartram High School,19142.0,None,...,0.293805,0.576991,0.033628,0.915044,0.030088,0.015929,0.003540,1.000000,0,1
3,ABDULALEEM,MUHAMMAD,70564,"TEACHER,FULL TIME",6090.0,CAREER AND TECHNICAL HIGH SCHL,M,A. Philip Randolph Career and Technical High S...,19129.0,None,...,0.187879,0.634343,0.002020,0.905051,0.016162,0.040404,0.026263,1.000000,0,0
4,ABDULLAH,AARON,46694,"TEACHER,FULL TIME",6360.0,ELEMENTARY SCHOOL,M,Theodore Roosevelt School,19144.0,Turnaround,...,0.178161,0.522989,0.001916,0.919540,0.040230,0.028736,0.009579,1.000000,1,0


In [58]:
full_turnover_2019.columns

Index(['LAST_NAME', 'FIRST_NAME', 'PAY_RATE_2018', 'TITLE_DESCRIPTION_2018',
       'HOME_ORGANIZATION', 'ORGANIZATION_LEVEL_2018', 'GENDER_2018', 'School',
       'Zip Code', 'Turnaround Model', 'Enrollment', 'Admissions Type',
       'Overall Score', 'Overall Tier', 'Ach Score', 'Ach Tier', 'Prog Score',
       'Prog Tier', 'Clim Score', 'Clim Tier', 'Attendance (95%+) Score',
       'Attendance (95%+) Tier', 'Retention Score', 'Retention Tier',
       'ISS Score', 'ISS Tier', 'OSS Score', 'OSS Tier',
       'Student Survey Climate Score', 'Student Survey Climate Tier',
       'Parent Survey Climate Score', 'Parent Survey Climate Tier',
       'Parent Survey Participation Score', 'Parent Survey Participation Tier',
       'Teacher Attendance Score', 'Student Survey Teaching Score',
       'Percent ELL', 'Percent IEP', 'Percent Male', 'Percent Asian',
       'Percent Black/African American', 'Percent Hispanic',
       'Percent Multi-Race', 'Percent White',
       'Economically Disadva

In [59]:
full_turnover_2019.rename(columns={'PAY_RATE_2018': 'PAY_RATE',
                   'TITLE_DESCRIPTION_2018': 'TITLE_DESCRIPTION',
                   'ORGANIZATION_LEVEL_2018': 'ORGANIZATION_LEVEL',
                   'GENDER_2018': 'GENDER'}, inplace=True)


In [60]:
full_turnover_2019.to_csv('data/df_2019.csv')

In [61]:
ls data/

2017-2018 Enrollment & Demographics.xlsx
SPR_SY1718_School_Metric_Scores_20190129.xlsx
df_2019.csv
df_eda.csv
employee_information_April2018.csv
employee_information_April2019.csv
